<a href="https://colab.research.google.com/github/BatoolMM/fastai/blob/master/chapter__7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uqq fastbook
!pip install fastcore==1.0.9
!pip install utils
!pip install wandb
!pip install nbdev

     |████████████████████████████████| 727kB 7.3MB/s 
     |████████████████████████████████| 194kB 39.0MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
     |████████████████████████████████| 1.0MB 43.4MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
     |████████████████████████████████| 92kB 9.9MB/s 
     |████████████████████████████████| 40kB 4.5MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 61kB 8.1MB/s 
     |████████████████████████████████| 2.6MB 41.0MB/s 
ERROR: nbdev 1.1.0 has requirement fastcore>=1.0.15, but you'll have fastcore 1.0.9 which is incompatible.
  Found existing installation: fastcore 1.0.19
    Uninstalling fastcore-1.0.19:
      Successfully uninstalled fastcore-1.0.19
     |████████████████████████████████| 1.7MB 6.8MB/s 
     |████████████████████████████████| 122kB 41.4MB/s 
     |████████████████████████████████| 102kB 10.7MB/

In [2]:
from utils import *
from fastbook import *
from fastai.vision.all import *
from fastai.vision.widgets import ImageClassifierCleaner
import wandb
from fastai.callback.wandb import WandbCallback
import matplotlib.pyplot as plt

In [3]:
path = untar_data(URLs.IMAGENETTE)

In [4]:
path = path/'val'
path.ls()

(#10) [Path('/root/.fastai/data/imagenette2/val/n01440764'),Path('/root/.fastai/data/imagenette2/val/n03394916'),Path('/root/.fastai/data/imagenette2/val/n03000684'),Path('/root/.fastai/data/imagenette2/val/n03425413'),Path('/root/.fastai/data/imagenette2/val/n02979186'),Path('/root/.fastai/data/imagenette2/val/n03888257'),Path('/root/.fastai/data/imagenette2/val/n03445777'),Path('/root/.fastai/data/imagenette2/val/n02102040'),Path('/root/.fastai/data/imagenette2/val/n03417042'),Path('/root/.fastai/data/imagenette2/val/n03028079')]

In [5]:
dblock = DataBlock(blocks=(ImageBlock(), CategoryBlock()),
                   get_items = get_image_files,
                   get_y = parent_label,
                   item_tfms = Resize(460),
                   batch_tfms = aug_transforms(size=224, min_scale=0.75))


In [6]:
dls = dblock.dataloaders(path, bs=64)

In [7]:
model = xresnet50()
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics = accuracy)
learn.fit_one_cycle(5,3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,2.569652,2.906499,0.285350,01:42
1,1.931079,2.598035,0.282803,01:43
2,1.591558,1.519148,0.491720,01:43
3,1.336620,1.114110,0.633121,01:43
4,1.128941,1.009954,0.668790,01:43


In [8]:
x, y = dls.one_batch()

In [9]:
x.mean(dim=[0,2,3])

TensorImage([0.4842, 0.4607, 0.4121], device='cuda:0')

In [10]:
x.std(dim=[0,2,3])

TensorImage([0.2725, 0.2625, 0.2911], device='cuda:0')

In [12]:
def get_dls(bs, size):
  dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                     get_items = get_image_files,
                     get_y = parent_label,
                     item_tfms = Resize(460),
                     batch_tfms = [*aug_transforms(size=size, min_scale=0.75),
                                   Normalize.from_stats(*imagenet_stats)])
  return dblock.dataloaders(path, bs=bs)

In [13]:
dls = get_dls(64, 224)

In [14]:
x, y = dls.one_batch()

In [15]:
x.mean(dim = [0,2,3]), x.std(dim =[0,2,3])

(TensorImage([-0.1273, -0.0191,  0.0955], device='cuda:0'),
 TensorImage([1.2697, 1.2474, 1.3153], device='cuda:0'))

In [16]:
model = xresnet50()
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(5, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,2.489431,4.029882,0.275159,01:43
1,1.894272,1.469310,0.528662,01:43
2,1.571134,1.302532,0.555414,01:43
3,1.313935,1.180358,0.621656,01:43
4,1.127803,1.016370,0.684076,01:43
